<a href="https://colab.research.google.com/github/ShirishaReddyV/AI-Sandbox/blob/main/Basic_RAG.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install langchain chromadb sentence-transformers PyMuPDF


In [ ]:
from google.colab import files
uploaded = files.upload()


In [ ]:
import fitz  # PyMuPDF
from langchain.schema import Document

def load_pdf_chunks(path, chunk_size=300):
    doc = fitz.open(path)
    text = ""
    for page in doc:
        text += page.get_text()

    chunks = [text[i:i+chunk_size] for i in range(0, len(text), chunk_size)]
    docs = [Document(page_content=chunk, metadata={"chunk_id": i}) for i, chunk in enumerate(chunks)]
    return docs

docs = load_pdf_chunks("small-business.pdf")  # update with your file path
print(f"✅ Loaded {len(docs)} chunks.")


In [ ]:
!pip install -U langchain-community

In [ ]:
from langchain.embeddings import HuggingFaceEmbeddings
from langchain.vectorstores import Chroma

embedding_model = HuggingFaceEmbeddings(model_name="all-MiniLM-L6-v2")
vectordb = Chroma.from_documents(docs, embedding=embedding_model, persist_directory="./safety_rag_chroma")
vectordb.persist()


In [ ]:
from langchain_community.embeddings import HuggingFaceEmbeddings
from langchain.vectorstores import Chroma

embedding_model = HuggingFaceEmbeddings(model_name="all-MiniLM-L6-v2")
vectordb = Chroma.from_documents(docs, embedding=embedding_model, persist_directory="./safety_rag_chroma")
vectordb.persist()

In [ ]:
def get_relevant_context(query, k=4):
    return vectordb.similarity_search(query, k=k)

query = "What are the emergency evacuation steps?"
relevant_docs = get_relevant_context(query)

context = "\n".join([doc.page_content for doc in relevant_docs])
print("📄 Retrieved Context:\n", context)


In [ ]:
from transformers import pipeline

qa_model = pipeline("text2text-generation", model="google/flan-t5-small", device="cpu")

prompt = f"""Answer the question based on the context below.

Context:
{context}

Question: {query}
"""

response = qa_model(prompt, max_new_tokens=150)
print("🤖 Answer:", response[0]['generated_text'].strip())
